In [1]:
import pandas as pd
import numpy as np
import keras
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import matplotlib.pyplot as plt

Using TensorFlow backend.
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\zippy\anaconda3\envs\te

In [2]:
# This function should transform data into a usable format 
def featureProcess(path):
    data = pd.read_csv(path)
    #get features data from loudness to vect148
    features = data.loc[:, "loudness":"vect_148"]

    return features.values.tolist()

In [3]:
# This function should transform data into a usable format 
def featureNormalProcess(path):
    data = pd.read_csv(path)
    #get audio data from vect1 to vect148
    #featureData = data.loc[:, "vect_1":"vect_148"]
    
    #get feature data from loudness to vect148
    featureData = data.loc[:, "loudness":"vect_148"]
    
    # normalization
    featureData=(featureData-featureData.mean())/featureData.std()
    
    return featureData.values.tolist()

In [4]:
# This function should transform data into a usable format 
def labelProcess(path):
    data = pd.read_csv(path)
    #get classes
    labels = data.loc[:,"genre"].values.tolist()
    
    return labels

In [5]:
def toOneHot(classes, labels):
    one_hot_labels = []

    for label in labels:
        one_hot_vector = [0] * len(classes)
        one_hot_vector[classes.index(label)] = 1

        one_hot_labels.append(one_hot_vector)
    return one_hot_labels

In [6]:
def MLP(trainData, trainLabel):
    
    num_classes = len(trainLabel[0])  # total classes to train
    
    model = keras.models.Sequential()
    model.add(Dense(trainData.shape[1], input_shape = trainData.shape[1:], activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.5))
    model.add(Dense(500, activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax", kernel_initializer="random_uniform"))

    #Define loss, optimizer, and metrics
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
def train(trainData, trainLabel, model):
    
    batch_size = 32  #the number of data to feed into model per batch
    num_epoch = 50  # go through your training data epoch times
    
    #callbacks stop training if val_loss is not improving.
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose = 1)]
    
    history = model.fit(trainData, trainLabel, validation_split=0.2, epochs = num_epoch, batch_size=batch_size, verbose=1, callbacks = callbacks)
                 
    return history, model

In [8]:
def evaluate(validData, validLabel, model):
    loss, accuracy = model.evaluate(validData, validLabel)
    return loss, accuracy

In [9]:
def predict(testData, model):
    result = model.predict_classes(testData, verbose=1)
    return result

DATA Preprocess

In [10]:
trainData = np.array(featureNormalProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_features.csv"))
validData = np.array(featureNormalProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_features.csv"))
testData = np.array(featureNormalProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv"))

In [11]:
trainLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_labels.csv")
validLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_labels.csv")
#get classes
classes = list(set(trainLabel))

oneHotTrain = np.array(toOneHot(classes, trainLabel))
oneHotvalid = np.array(toOneHot(classes, validLabel))

Training

In [12]:
model = MLP(trainData, oneHotTrain)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 148)               22052     
_________________________________________________________________
dropout_1 (Dropout)          (None, 148)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              149000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               500500    
_________________________________________________________________
dropout_3 (Dropout)  

In [13]:
history, _= train(trainData, oneHotTrain, model)
print("Finish training")

Instructions for updating:
Use tf.cast instead.
Train on 6142 samples, validate on 1536 samples
Epoch 1/50
6142/6142 [==============================] - 3s 503us/step - loss: 1.6371 - acc: 0.3758 - val_loss: 1.4093 - val_acc: 0.4674
Epoch 2/50
6142/6142 [==============================] - 1s 155us/step - loss: 1.4600 - acc: 0.4499 - val_loss: 1.3532 - val_acc: 0.5091
Epoch 3/50
6142/6142 [==============================] - 1s 141us/step - loss: 1.4197 - acc: 0.4728 - val_loss: 1.3367 - val_acc: 0.5111
Epoch 4/50
6142/6142 [==============================] - 1s 141us/step - loss: 1.3871 - acc: 0.4819 - val_loss: 1.3132 - val_acc: 0.5254
Epoch 5/50
6142/6142 [==============================] - 1s 147us/step - loss: 1.3491 - acc: 0.5049 - val_loss: 1.2773 - val_acc: 0.5378
Epoch 6/50
6142/6142 [==============================] - 1s 143us/step - loss: 1.3298 - acc: 0.5160 - val_loss: 1.2565 - val_acc: 0.5547
Epoch 7/50
6142/6142 [==============================] - 1s 141us/step - loss: 1.3155 - a

In [14]:
loss, accuracy = evaluate(validData, oneHotvalid, model)
print("loss, accuracy:", loss, accuracy)

450/450 [==============================] - 0s 49us/step
loss, accuracy: 1.6877009942796495 0.46444444444444444


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Predict

In [15]:
trackID = pd.read_csv(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv").loc[:, "trackID"].values.tolist()
prediction = predict(testData, model)

428/428 [==============================] - 0s 383us/step


In [16]:
results_name = []

for result in prediction:
    results_name.append(classes[result])

result = list(zip(trackID,results_name))

In [18]:
#file = "audio_MLP_outPut.csv"
file = "meta+audio_MLP_outPut.csv"
with open(file, 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(["trackID", "genre"])
    writer.writerows(result)